In [16]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

In [ ]:
role = get_execution_role() #get_execution_role recupera el rol de IAM que creó durante la creación de la instancia de notebooks
region = boto3.Session().region_name
bucket= 'lbk-analytics-dev' # Replace with your s3 bucket name
prefix = 'sagemaker/xgboost-mnist' # Used as part of the path in the bucket where you store data
bucket_path = 'https://lbk-analytics-dev.s3.amazonaws.com'

In [ ]:
container = get_image_uri(boto3.Session().region_name, 'xgboost' , '1.0-1')

In [ ]:
#Ensure that the train and validation data folders generated above are reflected in the "InputDataConfig" parameter below.
common_training_params = \
{
    "AlgorithmSpecification" : {
        "TrainingImage" : container,
        "TrainingInputMode" : "File"
    },
    "RoleArn" : role,
    "OutputDataConfig" : {
        "S3OutputPath" : bucket_path + "/" + prefix + "/xgboost"
    },
    "ResourceConfig" : {
        "InstanceCount" : 1 ,
        "InstanceType" : "ml.m4.xlarge" ,
        "VolumeSizeInGB" : 5
    },
    "HyperParameters" : {
        "max_depth" : "5" ,
        "eta" : "0.2" ,
        "gamma" : "4" ,
        "min_child_weight" : "6" ,
        "silent" : "0" ,
        "objective" : "multi:softmax" ,
        "num_class" : "10" ,
        "num_round" : "10"
    },
    "StoppingCondition" : {
        "MaxRuntimeInSeconds" : 86400
    },
    "InputDataConfig" : [
        {
            "ChannelName" : "train" ,
            "DataSource" : {
                "S3DataSource" : {
                    "S3DataType" : "S3Prefix" ,
                    "S3Uri" : bucket_path + "/" + prefix+ '/train/' ,
                    "S3DataDistributionType" : "FullyReplicated"
                }
            },
            "ContentType" : "text/csv" ,
            "CompressionType" : "None"
        },
        {
            "ChannelName" : "validation" ,
            "DataSource" : {
                "S3DataSource" : {
                    "S3DataType" : "S3Prefix" ,
                    "S3Uri" : bucket_path + "/" + prefix+ '/validation/' ,
                    "S3DataDistributionType" : "FullyReplicated"
                }
            },
            "ContentType" : "text/csv" ,
            "CompressionType" : "None"
        }
    ]
}

In [ ]:
#training job params
training_job_name = 'sagemaker-xgboost-' + strftime( "%Y-%m-%d-%H-%M-%S" ,gmtime())
print( "Job name is:" , training_job_name)
training_job_params = copy.deepcopy(common_training_params)
training_job_params[ 'TrainingJobName' ] = training_job_name
training_job_params[ 'ResourceConfig' ][ 'InstanceCount' ] = 1

In [15]:
%%time
region = boto3.Session().region_name
sm = boto3.Session().client( 'sagemaker' )
sm.create_training_job(**training_job_params)
status = sm.describe_training_job(TrainingJobName=training_job_name)[ 'TrainingJobStatus' ]
print(status)
sm.get_waiter( 'training_job_completed_or_stopped' ).wait(TrainingJobName=training_job_name)
status =sm.describe_training_job(TrainingJobName=training_job_name)[ 'TrainingJobStatus' ]
print( "Training job ended with status: " + status)
if status == 'Failed' :
    message =sm.describe_training_job(TrainingJobName=training_job_name)[ 'FailureReason' ]
    print( 'Training failed with the following error:{}' .format(message))
    raise Exception( 'Training job failed' )


's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-07-28 23:55:20 Starting - Starting the training job...
2020-07-28 23:55:23 Starting - Launching requested ML instances......
2020-07-28 23:56:41 Starting - Preparing the instances for training......
2020-07-28 23:57:43 Downloading - Downloading input data......
2020-07-28 23:58:52 Training - Downloading the training image...
2020-07-28 23:59:11 Training - Training image download completed. Training in progress.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Det